# Visualising $\Delta R$

The coordinate system is a modifed spherical coordinate system with the $z$ coordinate chosen along the beampipe, and the azimuthal angle $phi$ rotating around the plane perpendicular to the beam. In place of the usual polar angle $\theta$ however, pseudorapidity $\eta = -\ln(\tan(\frac{\theta}{2}))$ is used. See plot below to see how it behaves.

<img width="200" alt="preview" src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Pseudorapidity_plot.svg/695px-Pseudorapidity_plot.svg.png">

The angle $\Delta R$ is defined as $\Delta R = \sqrt{\Delta \phi^2 + \Delta \eta^2}$. It is tricky to grasp intuitively because the angle of this conical shape changes with $\eta$. This notebook makes it easy to play with parameters and see how the conical shape changes. 

First some imports

In [1]:
import numpy as np
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

Now define a helper function for handling the polar angle, and the function which draws the beampipe, an ATLAS-resembling cylinder, and the jet along with the conical shape around it, defined by dR.

I am using plotly instead of matplotlib because it is faster at rendering 3D.

In [2]:
def theta_from_eta(eta):
    return 2*np.arctan(np.e**(-eta))
    
def show_jet(phi, eta, dR):
    
    # list of all primitives to draw
    data = []
    
    # ATLAS beampipe
    R = 1.0
    zs = np.linspace(-R, R, 2)
    b_xs = [0 for z in zs]
    b_ys = [0 for z in zs]
    beampipe = go.Scatter3d(x=zs,
                          y=b_ys,
                          z=b_xs,
                          mode='lines',
                          showlegend=False,
                          line=dict(color='rgb(0,33,71)', width=5))
    data.append(beampipe)
    
    # ATLAS barrel
    phis = np.linspace(-np.pi/2., np.pi/2., 10)
    xs = R*np.sin(phis)
    X, Z = np.meshgrid(xs, zs)
    Y = np.sqrt(R**2 - X**2)
    barrel_left = go.Surface(x=Z, y=Y, z=X, surfacecolor=[1,1], showscale=False, opacity=0.4)
    barrel_right = go.Surface(x=Z, y=-Y, z=X, surfacecolor=[1,1], showscale=False, opacity=0.4)
    data += [barrel_left, barrel_right]
    
    # jet center line
    theta = theta_from_eta(eta)
    ts = np.linspace(0, R, 2)
    xs = ts*np.sin(theta)*np.cos(phi)
    ys = ts*np.sin(theta)*np.sin(phi)
    zs = ts*np.cos(theta)
    jet = go.Scatter3d(x=zs,
                       y=ys,
                       z=xs,
                       mode='lines',
                       showlegend=False,
                       line=dict(color='rgb(255, 105, 97)', width=7))
    data.append(jet)
    
    # jet cone: end circle and lines
    T = 1.0
    dphis = np.sqrt(np.linspace(0, dR**2, 20))
    detas = np.sqrt(dR**2 - dphis**2)
    # compute circle coordinates (reverse parts for plotting reasons)
    phis = np.concatenate((phi+dphis, phi+dphis[::-1], phi-dphis, phi-dphis[::-1]))
    etas = np.concatenate((eta+detas, eta-detas[::-1], eta-detas, eta+detas[::-1]))
    # transform to x, y, z
    thetas = theta_from_eta(etas)
    cxs = T*np.sin(thetas)*np.cos(phis)
    cys = T*np.sin(thetas)*np.sin(phis)
    czs = T*np.cos(thetas)
    cone_edge = go.Scatter3d(x=czs,
                            y=cys,
                            z=cxs,
                            mode='lines',
                            showlegend=False,
                            line=dict(color='rgb(255, 105, 97)', width=5))
    data.append(cone_edge)
    
    # and draw lines from each point to the origin
    for x, y, z in zip(cxs, cys, czs):
        line = go.Scatter3d(x=(0,z),
                            y=(0,y),
                            z=(0,x),
                            opacity=0.3,
                            mode='lines',
                            showlegend=False,
                            line=dict(color='rgb(255, 105, 97)'))
        data.append(line)
    
    # cosmetics
    layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0),
                       scene=dict(xaxis=dict(title='Z'),
                                  yaxis=dict(title='Y'),
                                  zaxis=dict(title='X')))
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='simple-3d-scatter')


Show a particular choice of $\phi$, $\eta$, and $\Delta R$, which renders in the nbviewer.

In [3]:
show_jet(0, 2.5, 1.0)

Unfortunately nbviewer does not display widgets, but the lines below should work if you download and run the notebook locally.

This should give you the option to change the values of $\phi$, $\eta$, and $\Delta R$ using sliders, as is shown in the preview image.

In [4]:
import ipywidgets as widgets
widgets.interact(show_jet, phi=widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=0.1, description='phi'),
                           eta=widgets.FloatSlider(value=2.0, min=0, max=3, step=0.1, description='eta'),
                           dR=widgets.FloatSlider(value=1.0, min=0.1, max=1.0, step=0.1, description='dR'))

interactive(children=(FloatSlider(value=0.0, description='phi', max=3.141592653589793, min=-3.141592653589793), FloatSlider(value=2.0, description='eta', max=3.0), FloatSlider(value=1.0, description='dR', max=1.0, min=0.1), Output()), _dom_classes=('widget-interact',))

<function __main__.show_jet>